In [ ]:
from dotenv import load_dotenv, find_dotenv
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_community.chat_models import ChatOllama
import requests
load_dotenv(find_dotenv())

In [ ]:
def fetch_model_names(base_url):
    api_endpoint = f"{base_url}/api/tags"
    try:
        response = requests.get(api_endpoint)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()
        model_names = [model["name"] for model in data.get("models", [])]
        return model_names
    except requests.RequestException as e:
        return str(e)

base_url = "https://ollama.junzis.com"
model_names = fetch_model_names(base_url)
model = model_names[0]
model

In [ ]:
llm = ChatOllama(base_url="https://ollama.junzis.com", model=model)
llm.invoke("hi")

In [ ]:
import base64
from PIL import Image
import io


def encode_image_to_base64(image_path):
    # Open the image file
    with Image.open(image_path) as image:
        # Convert the image to RGB format (to ensure compatibility)
        rgb_image = image.convert("RGB")
        # Create a BytesIO object to hold the image data
        buffered = io.BytesIO()
        # Save the image to the BytesIO object as JPEG (or PNG if you prefer)
        rgb_image.save(buffered, format="JPEG")
        # Retrieve the byte data from the buffer
        image_byte_data = buffered.getvalue()
        # Encode the byte data to Base64
        base64_encoded_image = base64.b64encode(image_byte_data)
        # Convert bytes to string
        base64_string = base64_encoded_image.decode("utf-8")
    return base64_string


# Example usage
image_path = "C:\\Users\\justa\\Downloads\\4-ac-conflict.jpg"  # Replace this with your image file path
base64_code = encode_image_to_base64(image_path)
print(base64_code)

In [ ]:
import base64
from io import BytesIO
from PIL import Image

def convert_to_base64(pil_image):
    """
    Convert PIL images to Base64 encoded strings

    :param pil_image: PIL image
    :return: Base64 string
    """
    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")  # You can change the format if needed
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser


# Function to create the prompt with text and image
def prompt_func(data):
    text = data["text"]
    image = data["image"]

    image_part = {
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{image}",
    }

    content_parts = []

    text_part = {"type": "text", "text": text}

    content_parts.append(image_part)
    content_parts.append(text_part)

    return [HumanMessage(content=content_parts)]

# Create the chain with the prompt function, model, and output parser
chain = prompt_func | llm | StrOutputParser()

# Load and convert the image to base64
file_path = "C:\\Users\\justa\\Downloads\\4-ac-conflict-larger.jpg"
pil_image = Image.open(file_path)
image_b64 = convert_to_base64(pil_image)

# Invoke the chain with the text and image data
query_chain = chain.invoke(
    {"text": "You are seeing the image of aircrafts in conflict. Orange means aircraft in conflict and green means it is not in conflict. Each arrow has an aircraft call sign, flight level FL and speed. Based on the image decide if there is an aircraft conflict and if so make a conflict resolution plan according to ICAO standards: at least 1000ft vertical seperation or 5 NM horizontal seperation", "image": image_b64}
)

print(query_chain)

In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cosine_similarity(list1, list2):
    # Convert lists to vectors
    vector1 = np.array(list1)
    vector2 = np.array(list2)

    # Calculate cosine similarity
    similarity = dot(vector1, vector2) / (norm(vector1) * norm(vector2))
    return similarity


# Example usage:
similarity_score = cosine_similarity(ac4, ac2_headon)
print(f"Cosine Similarity: {similarity_score}")

In [ ]:
similarity_score = cosine_similarity(ac2_headon_2, ac2_headon)
print(f"Cosine Similarity: {similarity_score}")

In [ ]:
model = OllamaFunctions(model="phi3:latest", format="json", keep_alive=-1)

In [ ]:
model.invoke('hi there!, tell why 2+2 is 5?')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_openai_tools_agent
import random
from langchain_core.tools import tool
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

ollama_model = ChatOpenAI(model="llama3.1:latest", base_url="http://localhost:11434/v1")
# ollama_model.invoke("hi")
gpt_model = ChatOpenAI(model="gpt-4o-mini")

@tool
def fake_weather_api(city: str) -> str:
    """Check the weather in a specified city. The API is available randomly, approximately every third call."""

    if random.randint(1, 3) == 1:  # Approximately one-third chance
        return "Sunny, 22°C"
    else:
        return "Service temporarily unavailable"


tools = [fake_weather_api]
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(gpt_model, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)
out = agent_executor.invoke({"input": "What is the weather in New York? and what is the weather in London?"})

out

In [ ]:
out

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_openai_tools_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
import random
from langchain_core.tools import tool
from langchain_core.tools import tool
import random
from langchain_core.utils.function_calling import convert_to_openai_function


@tool
def fake_weather_api(city: str) -> str:
    """Check the weather in a specified city. The API is available randomly, approximately every third call."""

    if random.randint(1, 3) == 1:  # Approximately one-third chance
        return "Sunny, 22°C"
    else:
        return "Service temporarily unavailable"


tools = [fake_weather_api]
functions = [convert_to_openai_function(tool) for tool in tools]
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(model, functions, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what is LangChain?"})

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.tools import tool


@tool
def get_weather(location: str) -> str:
    """
    Get the weather for a location

    """
    return "weather in " + location + " is sunny with a high of 75 degrees"

@tool
def get_population(location: str) -> str:
    """
    Get the population of a location

    """
    return "the population of " + location + " is 100,000"


tools = [get_weather, get_population]
functions = [convert_to_openai_function(t) for t in tools]
tool_mapping = {"get_weather": get_weather, "get_population": get_population}

In [ ]:
functions

In [ ]:
model = model.bind_tools(functions)

In [ ]:
response = model.invoke("what is the weather in Boston and its population?")
response

In [ ]:
response.additional_kwargs["function_call"]

In [ ]:
response.additional_kwargs | itemgetter("arguments")

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent